In [1]:
from datasets import load_dataset, concatenate_datasets
import json
import torch
from datasets.utils.logging import disable_progress_bar,enable_progress_bar

dataset_name = "super_glue"  # The MMLU dataset name in Hugging Face Datasets
dataset = load_dataset(dataset_name, 'copa')#["train"]
# print(dataset)

labels = dataset["train"].features["label"]
def create_label_str(batch):
    return {"label_str": labels.int2str(batch["label"])}
# print(labels)


/home/hshapour/.pyenv/versions/3.10.12/envs/pytorch/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Generating test split: 100%|██████████| 500/500 [00:00<00:00, 14218.27 examples/s]


In [2]:

data_train = dataset["train"].map(create_label_str)   
data_test = dataset["validation"].map(create_label_str)   

# d_all = concatenate_datasets([d1,d2])
# data_train = d_all.select(range(5))
# data_test = d_all.select(range(5,d_all.num_rows))
print(data_train.num_rows, data_test.num_rows)

# print(data_test)
# print(data_train)

# dd = data_test.select(range(5)).shuffle()
# print(data_train[4])
# print(data_test[0])


Map: 100%|██████████| 100/100 [00:00<00:00, 8055.90 examples/s]

400 100


In [3]:


def json_writer(JSON_PATH, num_qs, n_shot_example,seed=10,space=False):
    qlist = {}
    # if n_shot_example >0:
    #     instruction = 'The following are multiple choice questions (with answers) about  '
    # else:
    #     instruction = 'The following is a multiple choice question about  '
    with open(JSON_PATH, 'w') as json_file:

        for ind in range(num_qs):
            dd_examples = data_train.select(range(5)).shuffle()
            prompt = ''
            for idx in range(n_shot_example):
                prompt += "Premise: " + dd_examples[idx]['premise'] + '\n'
                prompt += "Question: " + dd_examples[idx]['question'] + '\n'
                prompt += "Alternative 1: " + dd_examples[idx]['choice1'] + '\n'
                prompt += "Alternative 2: " + dd_examples[idx]['choice2'] + '\n'
                prompt += "Answer:" + str(dd_examples[idx]['label']+1) + '\n\n'

            prompt += "Premise: " + data_test[ind]['premise'] + '\n'
            prompt += "Question: " + data_test[ind]['question'] + '\n'
            prompt += "Alternative 1: " + data_test[ind]['choice1'] + '\n'
            prompt += "Alternative 2: " + data_test[ind]['choice2'] + '\n'
            prompt += "Answer:"
    
            qlist['input'] = prompt
            qlist['target_str'] = data_test[ind]['label_str']
            qlist['target'] = data_test[ind]['label']

            # Serializing json
            json_file.write(json.dumps(qlist) + '\n')
        # print("Done!")

In [5]:
disable_progress_bar()

# print(example_idx)
n_example = data_test.num_rows
n_shot_vals = [2]
seed = 123
for n_shot in range(6):
# for n_shot in n_shot_vals:
    print(f"{n_shot}-shot example is being generated...")
    fname_json = f"copa-data/{n_shot}_shot_examples.json"
    # fname_json = f"boolq-data/{n_shot}_shot_examples_small.json"
    # JSON_eval = 'sample_eval.json'
    json_writer(fname_json, n_example, n_shot, seed=seed, space= False)
    # json_writer(JSON_eval,example_idx[(n_shot_example+1)*n_test_ex:])

enable_progress_bar()


0-shot example is being generated...
1-shot example is being generated...
2-shot example is being generated...
3-shot example is being generated...
4-shot example is being generated...
5-shot example is being generated...


In [35]:
# fname_json = f"mmlu-data/ex_no_space.json"
n_shot = 2
d1 = load_dataset("json", data_files={
        'dev' : f"copa-data/{n_shot}_shot_examples.json"
    })
# print(d1['dev'])
# print(d1['dev'].filter(lambda example: example["subject"] == subject))
# d2 = d1['dev'].filter(lambda example: example["subject"] == subject)
# # print(d2['input'][0][-1])
# # print("---------")
print(d1['dev'][2]['input'])

Premise: The runner wore shorts.
Question: cause
Alternative 1: The forecast predicted high temperatures.
Alternative 2: She planned to run along the beach.
Answer:1

Premise: My body cast a shadow over the grass.
Question: cause
Alternative 1: The sun was rising.
Alternative 2: The grass was cut.
Answer:1

Premise: The man got a discount on his groceries.
Question: cause
Alternative 1: He greeted the cashier.
Alternative 2: He used a coupon.
Answer:


In [6]:
from datasets import DatasetDict, Dataset
from collections import defaultdict

fname_dict = {}
for n_shot in range(6):
    fname_dict[f"{n_shot}_shot"] =  f"copa-data/{n_shot}_shot_examples.json"

# data_all = defaultdict(DatasetDict)
d_upload = load_dataset("json", data_files=fname_dict)
    # data_all[f"n_shot = {n_shot}"] = d1['dev']


Extracting data files: 100%|██████████| 6/6 [00:00<00:00, 1763.18it/s]
Generating 0_shot split: 100 examples [00:00, 20850.59 examples/s]
Generating 1_shot split: 100 examples [00:00, 36148.44 examples/s]
Generating 2_shot split: 100 examples [00:00, 48964.56 examples/s]
Generating 3_shot split: 100 examples [00:00, 38497.51 examples/s]
Generating 4_shot split: 100 examples [00:00, 28422.47 examples/s]
Generating 5_shot split: 100 examples [00:00, 32860.42 examples/s]


In [7]:
# print(Dataset.from_dict(data_all))
# Dataset.from_dict(data_all).push_to_hub("hassansh/boolq_n_shot")
d_upload.push_to_hub("hassansh/copa_n_shot")


Uploading the dataset shards: 100%|██████████| 1/1 [00:00<00:00,  1.21it/s]
README.md: 100%|██████████| 1.32k/1.32k [00:00<00:00, 3.29MB/s]
